# torchinfoでStable Diffusionの構造を見てみよう！

torchinfoのsummaryを使ってStable Diffusionの構造を見ていきます。ただしSkip connection等は表示されないので、これだけで完全に構造が分かるわけではありません。

入力次元を変えたりして出力がどう変わるかとかエラーが起きるかとかそういうのを実験できます。

ランタイム・・・None(CPU)＋標準で確認しています。

実装はhttps://yiskw713.hatenablog.com/entry/2021/06/01/070144
を参考にしています

In [ ]:
#ライブラリのインストール
!pip install --upgrade diffusers transformers scipy ftfy accelerate torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.9/524.9 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [ ]:
from diffusers import StableDiffusionPipeline
from torchinfo import summary
import torch

In [ ]:
#モデルのダウンロード
#SD_v1系とSD_v2系は形が違うよ、WDは現時点でv2
model_id = "hakurei/waifu-diffusion" #v1とv2以外ではモデルごとに違いはないはず

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    safety_checker = None #興味ないからNone
)

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/518 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/601 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
unet = pipe.unet
text_encoder = pipe.text_encoder
vae = pipe.vae

# text_encoder

プロンプトは、トークンという番号の列に置き換えられた後、text_encoderではプロンプトをAIが扱えるベクトルに変換されます。

トークンの数は77で、先端と終端を表す2つのトークン必要なので、実際に指定できるトークンは75個になります。トークンが足りない分は特殊なトークンで埋められます。

77個のトークン列が77×1024次元に置き換わります。
(※v1系は77×768次元になります)

実のところtext_encoderのことはよく分かってないですが、結果をみるとtransformersがいっぱい並んでいるようです。

In [ ]:
#batch size
batch_size = 1

#depthを少なくすると、浅い層だけに限定できます。最初は低くした方が分かりやすいかも。
depth = 10

#input_size、77個のトークン列(Long型)がバッチ分ある
#77を超えるトークン数を指定するとエラーが起こります。これがトークン数の限界になっています。
input_size=[(batch_size,77)]

summary(model=text_encoder, input_size=input_size,dtypes=[torch.int64],depth=depth,row_settings=["var_names"],col_names=["input_size","output_size","num_params"])

Layer (type (var_name))                                           Input Shape               Output Shape              Param #
CLIPTextModel (CLIPTextModel)                                     [1, 77]                   [1, 1024]                 --
├─CLIPTextTransformer (text_model)                                --                        [1, 1024]                 --
│    └─CLIPTextEmbeddings (embeddings)                            --                        [1, 77, 1024]             --
│    │    └─Embedding (token_embedding)                           [1, 77]                   [1, 77, 1024]             50,593,792
│    │    └─Embedding (position_embedding)                        [1, 77]                   [1, 77, 1024]             78,848
│    └─CLIPEncoder (encoder)                                      --                        [1, 77, 1024]             --
│    │    └─ModuleList (layers)                                   --                        --                        --
│    │    │    

# VAE

　 VAEは画像をよりデータ量が小さい潜在変数に置き換えてくれます。この潜在変数上で生成することで、生成時間やVRAM使用量を削減します。

## 入力
(batchsize,3(RGB),幅,高さ)になります。（幅と高さの順番は逆かもしれないけどどっちでもいいか）

 　通常は512×512画像を入れると思いますが、CPUだと時間がかかりすぎるので128×128を入れてみました。

 ## 出力
 (batch size,4,幅/8,高さ/8)になります。
 画像のサイズが64分の1になり、チャンネル数が4分の3になるので、データ量は48分の1になります。これってなんでチャンネルを1個増やすんですかね？誰か教えてください。

 　VAEはエンコーダ部分とデコーダ部分に分かれます。
 
 　エンコーダはまず(batch size,8,幅/8,高さ/8)にします。チャンネル数が8なのは、平均と分散がそれぞれ4チャンネルあるからです。そして平均と分散からサンプリングすることで、チャンネル数が4になります。

 　デコーダはエンコーダの出力を元の形に戻します。

## summary
 　結果をみると、基本的にはResNetが並んでいますが、面白いのはこの後でてくるUNetのモジュールを使いまわしていることですね。これはResNetとTransformersを組み合わせたblockになります。

In [ ]:
#batch size
batch_size = 1

#depthを少なくすると、浅い層だけに限定できます。最初は低くした方が分かりやすいかも。
depth = 10

#input size
input_size=(batch_size,3,128,128)

summary(model=vae, input_size=input_size,depth=10,row_settings=["var_names"],col_names=["input_size","output_size","num_params","kernel_size"])

Layer (type (var_name))                                 Input Shape               Output Shape              Param #                   Kernel Shape
AutoencoderKL (AutoencoderKL)                           [1, 3, 128, 128]          [1, 3, 128, 128]          --                        --
├─Encoder (encoder)                                     [1, 3, 128, 128]          [1, 8, 16, 16]            --                        --
│    └─Conv2d (conv_in)                                 [1, 3, 128, 128]          [1, 128, 128, 128]        3,584                     [3, 3]
│    └─ModuleList (down_blocks)                         --                        --                        --                        --
│    │    └─DownEncoderBlock2D (0)                      [1, 128, 128, 128]        [1, 128, 64, 64]          --                        --
│    │    │    └─ModuleList (resnets)                   --                        --                        --                        --
│    │    │    │    └─Resne

#UNet

 Stable Diffusionのメインのネットワークになります。計算時間のほとんどがこの部分になります。（そうなるのは1回の生成で何度も繰り返さなければいけないからで、今回は1回切りなのでそこまで時間かかりません）

 UNetの目的はノイズを除去することになります。ノイズ除去を極めた結果完全なノイズから画像を作れるようになっちゃったというイメージです。ノイズは一気に除去するのではなく少しずつ除去します。そのため何度もUNetの計算を繰り返す必要があります。

## 入力

 ノイズ付きの潜在変数：形はVAEのエンコーダ部分の出力と同じ (batch size,4,幅/8,高さ/8)
 
 ステップ数：繰り返しの中で、現在何回目かを教えるための入力です。 (1,)

 テキストの埋め込みベクトル：テキストエンコーダの出力 (batch size,77,1024)

 の3つが必要になります。

## 出力
元の潜在変数から予測したノイズ(v_paramの場合違うけど)になります。

## summary
UNetはdown_blocks, mid_block, up_blocksに分かれます。donw_blocksでは画像のサイズを縮めながらチャンネル数を増やしていきます。up_blocksはその逆です。この結果では見ることができませんが、skip connectionがいっぱいあります。またtime_embがResNetに入力されているといった仕組みもこの結果だけでは見ることができません。

基本的にはResNetBlockとTransformersが交互にならんでいます。TransformersはAttentionが2個とFeedForwardで作られています。Attentionの1個目はSelfAttentionであり、2個目はCrossAttentionになっています。テキストの埋め込みベクトルはCrossAttentionに入力されます。

## 発展
　画像サイズはdown_blocksによる3回のdown_sampleによって8分の1になることが分かります。VAEと合わせれば画像サイズは64分の1になります。これがStable Diffusionで生成する画像の解像度を64刻みにする理由になります。

　UNet部分にはトークン数の制限がありません。そのおかげでトークン数の制限をなくすことができます。たとえばトークン数を3倍にしたいときは、トークン列を3分割して、それぞれtext encoderに入れて、結果を横に並べることで、埋め込みベクトル(batch size,227,1024)を作ることができます。77⇒227になっても。to_kやto_vは計算できますし、$qk^Tv$部分のサイズは変わらないためエラーが起きません。でもこんなことしてうまくいくのは不思議です。

　画像を生成するときは、Classifier Free Guidance(CFG)という手法を利用するため、プロンプトを空文に置き換えた際の出力も必要になります。その場合はbatch sizeが2倍になったかのように扱われます。


In [ ]:
#batch size
batch_size = 1

#depthを少なくすると、浅い層だけに限定できます。最初は低くした方が分かりやすいかも。
depth = 10

#CFGを使う場合2、使わない場合1
cfg = 1

#潜在変数サイズ
latent_size = (batch_size * cfg,4,32,32)

#ステップ数
time_step = (1,)

#テキストの埋め込みベクトル
tokens = 75 * 3
text_embeddings = (batch_size * cfg,tokens + 2,1024)


summary(model=unet, input_size=[latent_size,time_step,text_embeddings],depth=depth,row_settings=["var_names"],col_names=["input_size","output_size","num_params","kernel_size"])

Layer (type (var_name))                                           Input Shape               Output Shape              Param #                   Kernel Shape
UNet2DConditionModel (UNet2DConditionModel)                       [1, 4, 32, 32]            [1, 4, 32, 32]            --                        --
├─Timesteps (time_proj)                                           [1]                       [1, 320]                  --                        --
├─TimestepEmbedding (time_embedding)                              [1, 320]                  [1, 1280]                 --                        --
│    └─Linear (linear_1)                                          [1, 320]                  [1, 1280]                 410,880                   --
│    └─SiLU (act)                                                 [1, 1280]                 [1, 1280]                 --                        --
│    └─Linear (linear_2)                                          [1, 1280]                 [1, 1280]       